In [1]:
import pandas as pd
from pandas import merge
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Procesamiento

A continuación estan todas las funciones para generar la bolsa de palabras

## Limpieza de datos

In [2]:
def limpiar(oracion):
    oracion = oracion.lower()
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
    return oracion

## Tokenización

In [3]:
def tokenizacion(oracion):
    #nltk.download('punkt')
    tokens = word_tokenize(oracion)
    return tokens

## Stopwords

In [4]:
#nltk.download('stopwords')
def stopword(tokens):
    spanish_stopwords = stopwords.words('spanish')
    palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]
    return palabras_filtradas

## Lematización

In [5]:
#sudo python3 -m spacy download es
def lematización(palabras_filtradas):
    nlp = spc.load("es_core_news_sm")
    lema = nlp(" ".join(palabras_filtradas))
    oracion_lematizada = " ".join([token.lemma_ for token in lema])
    return oracion_lematizada

## Bag of Words

In [6]:
def generar_bolsa(oraciones_lematizadas):
    vectorizador = CountVectorizer()
    vectores = vectorizador.fit_transform(oraciones_lematizadas)
    vocabulario = vectorizador.get_feature_names_out()
    return vocabulario, vectores

# Trabajando con el conjunto de datos

In [7]:
df_words = pd.read_csv(f"df_mini_HS.csv")
df_words

,label,text
0,1,ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1,1,"28: te amodio, odio a la perra de tu amiga ☺️☺..."
2,1,@LaDivinaDiva Callate maldita perra. O seguro ...
3,1,@MarysabelPuerto Mejor callate cara de puta o ...
4,1,@xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...
5,1,@CocotePR @ashleyhonohan callate puta
6,1,Y el inmigrante recibe ayuda del rico Estado l...
7,1,"De los moros no se puede esperar nada bueno, y..."
8,1,¿Por que si a una mujer le pegan un tiro en la...
9,1,"Analicemos esto: ¿Si te pones unos shorts así,..."


In [8]:
labels = df_words['label'].tolist()
oraciones_lematizadas = []
for i in range(0, len(df_words)):
    oracion = limpiar(df_words.at[i, 'text'])
    tokens = tokenizacion(oracion)
    palabras_filtradas = stopword(tokens)
    oracion_lematizada = lematización(palabras_filtradas)
    oraciones_lematizadas.append(oracion_lematizada)

In [11]:
vocabulario, vectores = generar_bolsa(oraciones_lematizadas)
df_bw = pd.DataFrame.sparse.from_spmatrix(vectores, columns=vocabulario)
df_bw['label'] = labels
#df_bw.to_csv('df_bw.csv')
df_bw

,acoso,agar,agresión,amigo,amodio,analicer,aprieto,arab,arar,así,...,vez,vida,viola,voolka,vía,yogurines,you,árabe,él,label
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,0,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
